This notebook shows showcases of how to use every measurement function in cqed.custom_pysweep_functions.vna.py. It contains two classes of functions: functions that use the VNA in linear mode and functions that use the VNA in CW mode. 





# Imports

In [ ]:
import pysweep
import pysweep.databackends.debug
import pysweep.databackends.list_backend
import pysweep.databackends.qcodes
import pysweep.core.measurementfunctions
from pysweep.core.sweepobject import SweepObject
from pysweep.core.measurementfunctions import MakeMeasurementFunction
from pysweep.databackends.base import DataParameterFixedAxis
from pysweep.databackends.base import DataParameter
import pysweep.convenience as conv

import qcodes as qc
from qcodes.dataset.measurements import Measurement
import qcodes.dataset.plotting

from cqed.utils.datahandling import create_local_dbase_in
import cqed.custom_pysweep_functions.vna as cvna
import cqed.custom_pysweep_functions.magnet as cmgnt

import numpy as np
import matplotlib.pyplot as plt

# Set up station and QCoDeS database

## Set up measurement station

Choose the appropriate file for your measurement setup. The repo contains templates with commonly used instruments for K1 and K2.

In [ ]:
station = qc.Station(config_file=r'C:\code\environments\qcodes_dev\cqed\station_init\LK2.station.yaml')

## Set up instruments

For this example we are only going to be using the ZNB20 VNA from R&S.

In [ ]:
vna = station.load_instrument('vna')

vna.add_channel('S21')
station.vna.S21.power(-50)
station.vna.S21.start(4e9)
station.vna.S21.stop(8.5e9)
station.vna.S21.bandwidth(1e3)
station.vna.S21.npts(10001)

# Automatically remove the cable delay from the phase signal
# Should be checked in large frequency range and with higer readout power
# Often can be fine-tuned a bit better by hand.
station.vna.S21.set_electrical_delay_auto()

## Pass measurement station to pysweep 


In [ ]:
pysweep.STATION = station

## Name and initialize database

In [ ]:
# To place the database in D:/Data/MyExperiment/ use:
folder = 'MyExperiment'

# To place the database in a more nested folder structure, e.g. D:/Data/MyExperiment/TestSample/FirstMeasurements/, use:
# folder = 'MyExperiment/TestSample/FirstMeasurements'

# Name your database
database_name = 'yyyy_mm_dd_Run1'

create_local_dbase_in(folder_name=folder, db_name='%s.db' %(database_name))

qc.config['core']['db_location']

# Linear VNA Mode

Linear mode is the main mode of the VNA, where it measures S parameters over a certain frequency range. This is the mode we will use most often, for example for studying resonators.

##  Single VNA trace

This just records a single VNA trace with the VNA parameters currently set or set (if specified) in the init_measurement function.

In [ ]:
# Function called before beginning of measurement loop.
# This is a good place to define the VNA sweep parameters. Can also be made a function with changable parameters.

def init_measurement(d):
    # some example parameters that could be set in the init function
    station.vna.rf_on()
    station.vna.S21.start(4e9)
    station.vna.S21.stop(5e9)
    station.vna.S21.npts(1001)
    
    station.vna.S21.power(-20)
    station.vna.S21.bandwidth(1e3)
    station.vna.S21.avg(10)
    
    print('Starting measurement.')
    
# Function called after the end of the measurement.
def end_measurement(d):
    print('Measurement finished.')

# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='single_VNA_trace', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cvna.measure_linear_sweep(),
                 databackend = pysweep.databackends.qcodes.DataBackend(meas))

For plotting of the generated dataset in the notebook directly

In [ ]:
qc.dataset.plotting.plot_dataset(result.datasaver.dataset)

Alternatively to using the init_measurement function, one can also specify parameters to be ussed during the measurement in measure_linear_sweep itself through keyword arguments such as they appear in 'cvna.setup_vna'. Note that parameters are only set such keyword arguments are present, so this functionality does not incur any speed penalty.

In [ ]:
def init_measurement(d):
    station.vna.rf_on()
    station.vna.S21.start(4e9)
    station.vna.S21.stop(5e9)
    station.vna.S21.npts(1001)
    station.vna.S21.power(-20)
    station.vna.S21.bandwidth(1e3)
    station.vna.S21.avg(10)
    
    print('Starting measurement.')
    
def end_measurement(d):
    print('Measurement finished.')

sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='single_VNA_trace', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cvna.measure_linear_sweep(center=5e9, span=10e6, npts=1001),
                 databackend = pysweep.databackends.qcodes.DataBackend(meas))

In [ ]:
qc.dataset.plotting.plot_dataset(result.datasaver.dataset)

Even though we set a range of 4 to 5 GHz in the init_measurement, these settings were overwritten upon actually executing the measurement function. You may ask why you want to do this. an example of that is measuring multiple resonators in the same measurement. 

## Measuring multiple VNA traces

We have two ways of measuring N VNA traces in the same measurement. The first is through pysweeps addition function and the use of the optional suffix argument

In [ ]:
exp = qc.load_or_create_experiment(experiment_name='multiple_VNA_traces_suffix', sample_name=sample_name)
meas = Measurement(exp, station)

meas_res_0 = cvna.measure_linear_sweep(suffix='_0', center=5e9, span=10e6, npts=1001)
meas_res_1 = cvna.measure_linear_sweep(suffix='_1', center=6e9, span=10e6, npts=1001)

result = pysweep.sweep(init_measurement, end_measurement, meas_res_0 + meas_res_1,
                 databackend = pysweep.databackends.qcodes.DataBackend(meas))

This procedure can become a little cumbersome when one has many different traces to measure. For this reason we have the helper function 'measure_multiple_linear_sweeps' which allows the user to give lists or arrays of parameter arguments, from which the concatenation of measurement functions is then automatically created.

In [ ]:
# Set centerfrequencies for individual resonators
center_list = [5.0e9, 5.5e9, 6.0e9, 6.5e9]
# Set measurement spans for individual resonators
span_list = [20e6, 20e6, 20e6, 20e6]

exp = qc.load_or_create_experiment(experiment_name='multiple_VNA_traces_helperfun', sample_name=sample_name)
meas = Measurement(exp, station)

meas_multiple = eval(cvna.measure_multiple_linear_sweeps(center_list=center_list, span_list=span_list))

result = pysweep.sweep(init_measurement, end_measurement, meas_multiple,
                 databackend = pysweep.databackends.qcodes.DataBackend(meas))

## Parameter sweep

The next thing one might like to do is measure a VNA sweep versus a QCoDeS parameter, such as the VNA power. This proceeds as follows:

In [ ]:
exp = qc.load_or_create_experiment(experiment_name='VNA_vs_pwr', sample_name=sample_name)
meas = Measurement(exp, station)


result = pysweep.sweep(init_measurement, end_measurement, cvna.measure_linear_sweep(),
                       pysweep.sweep_object(station.vna.S21.power, np.arange(-50, -15, 5)),
                       databackend = pysweep.databackends.qcodes.DataBackend(meas))

Be careful that if one had provided the keyword argument 'pwr' to 'measure_linear_sweep' this would have overwritten the power set by the sweep object.

Such sweeps also work when measuring multiple VNA traces:

In [ ]:
exp = qc.load_or_create_experiment(experiment_name='multiple_VNA_traces_vs_power', sample_name=sample_name)
meas = Measurement(exp, station)

meas_res_0 = cvna.measure_linear_sweep(suffix='_0', center=5e9, span=10e6, npts=1001)
meas_res_1 = cvna.measure_linear_sweep(suffix='_1', center=6e9, span=10e6, npts=1001)

result = pysweep.sweep(init_measurement, end_measurement, meas_res_0 + meas_res_1,
                       pysweep.sweep_object(station.vna.S21.power, np.arange(-50, -15, 5)),
                       databackend = pysweep.databackends.qcodes.DataBackend(meas))

## Finding resonances in the VNA sweep

Oftentimes in our measurements the VNA will be used to find resonances, and frequently the measurement that comes after depends on where that resonance frequency was. An example of that is two-tone spectroscopy, which we return to in the CW section. Our vna module offers a function that wraps around 'measure_linear_sweep' to find resonancse and store their values in the pysweep waterfall dictionary. This can proceed as follows:

We first import or build a peak finding function

In [ ]:
from cqed.utils.peak_finding import find_peak_literal
resonance_finder = lambda x,y : find_peak_literal(x, y, dip=True, dBm=True)

In [ ]:
exp = qc.load_or_create_experiment(experiment_name='VNA_trace_with_peakfinding', sample_name=sample_name)
meas = Measurement(exp, station)


find_resonator = cvna.measure_resonance_frequency(peak_finder=resonance_finder, save_trace=True, center=6.055e9, span=50e6, fstep=0.2e6, navgs=10, bw=1e3, pwr=-37)

result = pysweep.sweep(init_measurement, end_measurement, find_resonator,
                 databackend = pysweep.databackends.qcodes.DataBackend(meas))

In addition to the VNA trace itself (which was saved because save_true=True was used), the dataset now also contains the resonance frequency:

In [ ]:
d = qc.load_by_id(result.datasaver.run_id)
resonance_frequency = d.get_parameter_data('resonance_frequency')['resonance_frequency']['resonance_frequency'][0]
print(resonance_frequency)

Why is this useful? That resonance frequency can now be used by other measurement functions in the same measurement. An example of where that comes in handy is in adaptive measurements.

## Adaptive VNA measurements

It frequently comes up that one would like to measure a resonance frequency versus a control parameter that is able to change that frequency, such as a gate voltage or the magnetic field. But one does not always know in what direction or over what range the frequency might move during the measurement. One way of dealing with that is measuring over a very large range. This will work, but it means you spend most of your measurement time on featureless backgrounds. Instead, one can choose for an adaptive measurement strategy. 

We have implemented a naive first approach to doing so, but one can think of a large number of more complicated methods. So far it has not yet been neccesary, but we have a number of ideas. 

The method we currently support is first doing a fast, broad frequency range scan from which you find the resonance, followed by a slow, zoomed in scan around that resonance. This proceeds as follows

In [ ]:
exp = qc.load_or_create_experiment(experiment_name='adaptive_VNA_trace', sample_name=sample_name)
meas = Measurement(exp, station)


find_resonator = cvna.measure_resonance_frequency(peak_finder=resonance_finder, save_trace=True, suffix='_coarse', center=6.055e9, span=35e6, fstep=0.5e6, navgs=2, bw=1e3, pwr=-30)
measure_resonator = cvna.measure_adaptive_linear_sweep(suffix='_fine', span=10e6, fstep=0.1e6, navgs=10, bw=1e3, pwr=-37)

result = pysweep.sweep(init_measurement, end_measurement, find_resonator + measure_resonator,
                 databackend = pysweep.databackends.qcodes.DataBackend(meas))

# CW VNA mode

Continuous wave (CW) mode is a second mode of the VNA that we frequently use. In those mode, the VNA frequency is not swept, but instead the transmission is read out N times at a single frequency with a certain bandwidth. Two example cases where this comes in handy are two-tone spectroscopy and time traces. 

## Two tone spectroscopy

In a nutshell, two tone spectroscopy involves monitoring the signal for a fixed frequency of the first tone while the frequency of a second tone is swept. Our modules offer two ways of doing so, which we showcase in this section. 

First we are going to need the instrument we use for the second tone, which we name the qubsrc (qubit source)

In [ ]:
qubsrc = station.load_qubsrc()
qubsrc.modulation_rf('OFF')
#we add some custom named parameters as their names can conflict with those of the VNA
qubsrc.add_parameter('drive_freq', set_cmd=qubsrc.frequency,
                    get_cmd=None, unit='Hz')
qubsrc.add_parameter('drive_power', set_cmd=qubsrc.power,
                    get_cmd=None, unit='dBm')




The first method of two-tone spectroscopy simply sets up the VNA in CW mode, measures a mag-phase pair using 'return_cw_point' and then sweeps the frequency of a second tone as a pysweep object.

In [ ]:
def init_measurement(d): 
    t_int = 100e-3
    bw = 1e2
    station.vna.rf_on() 
    station.vna.S21.setup_cw_sweep()
    station.vna.S21.cw_frequency(resonance_frequency)
    station.vna.S21.bandwidth(bw)
    station.vna.S21.npts(int(np.round(t_int*bw)))
    station.vna.S21.power(-47)
    qubsrc.power(-45)
    qubsrc.output_rf('ON')
    qubsrc.modulation_rf('OFF')
    print('Starting measurement.')

def end_measurement(d):
    qubsrc.output_rf('OFF')
    station.vna.rf_off()
    print('Measurement finished.')

sample_name = 'sample1'
exp = qc.load_or_create_experiment(experiment_name='2tone_trace_pysweep', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, 
                       cvna.measure_cw_point(),
                       pysweep.sweep_object(qubsrc.drive_freq, np.arange(3.5e9, 5.5e9, 5e6)),
                       databackend = pysweep.databackends.qcodes.DataBackend(meas))



Note that instead of using the init we could once more have specified the VNA parameters as keyword arguments to 'return_cw_point' (see its documentation with cvna.return_cw_point?)

## Two tone spectroscopy as a meta instrument

A second way of doing two-tone spectroscopy involves 'measure_twotone_sweep', which is a measurement function that acts as a sort of metainstrument encompassing both the VNA and the qubsrc. Essentially the sweep of the second tone is implicitly included in the measurement itself. This is mainly useful if your twotone measurement is just one step in a longer chain, where the later steps require you to extract quantities from the twotone result. We will see an example of how this works a few cells down. This also helps getting around that pysweep only allows 3D scans (while we might want to sweep frequency, gate, flux and field in a single measurement, for example) but that is less common.

Note that for the future I would actually like to create a proper QCoDeS metainstrument that makes things a bit cleaner, but I have not done so yet. The current implementation works as follows

In [ ]:
t_int = 100e-3
bw = 1e2
npts = int(np.round(t_int*bw))

def init_measurement(d): 
    station.vna.rf_on() 
    station.vna.S21.setup_cw_sweep()
    qubsrc.output_rf('ON')
    qubsrc.modulation_rf('OFF')
    print('Starting measurement.')

def end_measurement(d):
    qubsrc.output_rf('OFF')
    station.vna.rf_off()
    print('Measurement finished.')

sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='2tone_trace_selfcontained', sample_name=sample_name)
meas = Measurement(exp, station)

find_resonator = cvna.measure_resonance_frequency(peak_finder=resonance_finder, suffix='_0', save_trace=True, center=6.055e9, span=50e6, fstep=0.2e6, navgs=10, bw=1e3, pwr=-37)


result = pysweep.sweep(init_measurement, end_measurement, 
                       find_resonator + cvna.measure_twotone_sweep(frequencies=np.arange(3.5e9, 5.5e9, 5e6), qubsrc_power=-35, suffix='_1', bw=bw, npts=npts, pwr=-47),
                       databackend = pysweep.databackends.qcodes.DataBackend(meas))

## Two tone spectroscopy with peak finding

As we mentioned, this measurement function mainly shines when you want to do something with the outcome of the twotone measurement result. An example is finding the qubit frequency; this is implemented in analogy with finding the resonator frequency

In [ ]:
qub_finder = lambda x,y : find_peak_literal(x,y,dip=False, dBm=True)

In [ ]:
def init_measurement(d):    
    print('Starting measurement.')


def end_measurement(d):
    qubsrc.output_rf('OFF')
    station.vna.rf_off()
    print('Measurement finished.')

sample_name = 'sample'

exp = qc.load_or_create_experiment(experiment_name='find_resonator_and_qubit', sample_name=sample_name)
meas = Measurement(exp, station)

find_resonator = cvna.measure_resonance_frequency(peak_finder=resonance_finder, suffix='_res', save_trace=True, center=6.055e9, span=50e6, fstep=0.2e6, navgs=10, bw=1e3, pwr=-37)
find_qubit = cvna.measure_qubit_frequency(frequencies=np.arange(3.5e9, 5.5e9, 5e6), qubsrc_power=-35, 
                                          suffix='_qub', save_trace=True, peak_finder=qub_finder, bw=1e2, npts=10, pwr=-47)

result = pysweep.sweep(init_measurement, end_measurement, 
                       find_resonator + find_qubit,
                       databackend = pysweep.databackends.qcodes.DataBackend(meas))
d = qc.load_by_id(result.datasaver.run_id)
qubit_frequency = d.get_parameter_data('qubit_frequency_qub')['qubit_frequency_qub']['qubit_frequency_qub'][0]
print(qubit_frequency)

## Time traces

A second usecase of the VNA in CW mode is taking time traces. Think of studying time resolved phenomena quasiparticle poisoning, but also debugging purposes such as looking at the power spectral density of your measurement signal.

In [ ]:
def init_measurement(d): 
    print('Starting measurement.')
def end_measurement(d):
    print('Measurement finished.')

sample_name = 'sample'
exp = qc.load_or_create_experiment(experiment_name='time_trace', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, 
                       cvna.measure_cw_sweep(cw_frequency=resonance_frequency, t_int=10, bw=1e4, pwr=-37),
                       databackend = pysweep.databackends.qcodes.DataBackend(meas))



One can look at the time trace in the IQ plane

In [ ]:
data = qc.load_by_id(result.datasaver.run_id)
I = data.get_parameter_data('I')["I"]["I"][0]
Q = data.get_parameter_data('Q')["Q"]["Q"][0]
times = data.get_parameter_data('Q')["Q"]["time"][0]

In [ ]:
plt.figure()
plt.hist2d(I,Q, bins=(50, 50), cmap='Greys')
plt.plot()

Or instead look at the power spectral density

In [ ]:
ys_fft = np.abs(np.fft.fft((Q - np.mean(Q))+1j*(I - np.mean(I))))**2
dt = np.diff(times)[0]
fmax = 1. / (dt)
df = 1. / (len(times)*dt)
pts_fft = int(len(times) / 2)
fs1 = np.linspace(0, fmax / 2.-df, pts_fft)
fs2 = np.linspace(fmax / 2., df, pts_fft)
fs = np.append(fs1, -fs2)

plt.figure()
plt.plot(fs[1:pts_fft], ys_fft[1:pts_fft])
plt.yscale('log')
plt.xscale('log')

## Averaged PSD measurements

In particular for the fft one might want to average a number of traces together to improve the SNR. For this we also have a convenience function available. It uses a rolling average so only 2 traces are ever stored in the memory; this means you don't have to be afraid of memory overflow when for example trying to average 1000 traces of 1e5 points each.

In [ ]:
def init_measurement(d): 
    print('Starting measurement.')
def end_measurement(d):
    print('Measurement finished.')

sample_name = 'sample'
exp = qc.load_or_create_experiment(experiment_name='averaged_PSD', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, 
                       cvna.measure_PSD_averaged(averages=10, cw_frequency=4.5e9, t_int=10, bw=1e4, pwr=-37),
                       databackend = pysweep.databackends.qcodes.DataBackend(meas))



## SNR Measurements

The final function we have available for the CW mode is particularly useful for amplifier characterisation, such as for the TWPA. It allows one to measure the signal (mean), noise (std) and SNR (mean/noise) at a fixed frequency versus QCoDeS parameters of choice.

In [ ]:
t_int = 50e-3
bw=1e4
npts = int(np.round(t_int*bw))

def init_measurement(d): 
    station.TWPA.on()
def end_measurement(d):
    station.TWPA.off()

sample_name = 'sample'
exp = qc.load_or_create_experiment(experiment_name='TWPA_optimization', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, 
                       cvna.measure_SNR_CW(t_int=t_int, bw=bw, pwr=-38, cw_frequency=6e9),
                       pysweep.sweep_object(TWPA_freq, np.arange(6.75e9,8.4e9,5e6)),
                       pysweep.sweep_object(TWPA_power, np.arange(-25, -10, 0.5)),
                       databackend = pysweep.databackends.qcodes.DataBackend(meas))